In [ ]:
# Mosaic + Backfill + Clip 270m FSim results at landscape level#
# mlazarz 03/02/24

# Backfill section comes from Julia's 'Backfill_FLP_270m_rastersNatl2023' script
# Extent and mask ajustments were made for landscape backfilling.

import arcpy 
import os
from itertools import repeat
from arcpy import env
from arcpy.sa import *
arcpy.ResetEnvironments()
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True

# Band order:  1- BP, 2- FLP1, 3- FLP2, 4- FLP3, 5- FLP4, 6- FLP5, 7- FLP6, 8- CFL (not needed)

# function to create folders
def checkPath(path):
    try: os.makedirs(path)
    except: pass

# Landscape name (for output folder name) and pyromes needed to fill landscape
landscape = "Stanislaus"
pyromes_needed = ['026', '027', '028', '029', '030', '078'] # Use string format for query from outputs folder 

# Final extent #

# landscape buffer (60 km) - I split this out beforehand
#### update this to the Analysis_Area_60km (updated wcs landscapes)
wcs_buffer = r"D:\d_WCS_OPM_2024\2_Fuelscapes\Inputs\wcs_buffered\buffer_by_landscape.gdb\Stanislaus"

# Mosaic Inputs #

# folder where 270m Fsim multi-band outputs are located by pyrome
fsim270 = r"D:\d_WCS_OPM_2024\4_FSim_Outputs\Post-Treatment\270m_Outputs"
# lookup table with number of iterations
iteration_lut_path = r"C:\Data\WCS_OPM_2024\3_FSim_Inputs\cmdx_files\full_lookup.csv"

# Backfill Inputs #

# Directory with mosaicked fuel rasters (we need the fuelscape that matches FSim run)
fuelsDir = r"C:\Data\WCS_OPM_2024\2_Fuelscapes\Output\Fuels_Mosaics_270m"

# Fuel raster base names
FM40 = "S3_40_270m.tif"
CC = "S3_CC_270m.tif"
CBH = "S3_CBH_270m.tif"

# Pyromes layer
pyromesRaster = r"C:\Data\WCS_OPM_2024\2_Fuelscapes\FSim_2022\270m\Extents\Extent_CONUS_PYROME_270m.tif"

# Path to snap raster
snapRas = r"C:\Data\WCS_OPM_2024\2_Fuelscapes\FSim_2022\270m\Extents\Extent_CONUS_270m.tif"

# Set environments
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = snapRas
arcpy.env.snapRaster = snapRas
arcpy.env.cellSize = snapRas
arcpy.env.compression = "LZW"
#arcpy.env.mask = pyromesRaster  # Otherwise it will extend into Canada (Not needed)

# Location for outputs
output_folder = r"D:\d_WCS_OPM_2024\4_FSim_Outputs\Post-Treatment\270m_Outputs_By_Landscape"
checkPath(output_folder)
# Location for landscape outputs
landscape_folder = os.path.join(output_folder, landscape)
checkPath(landscape_folder)
# Mosaic folder
mosaic_folder = os.path.join(landscape_folder , "Mosaic")
checkPath(mosaic_folder)
# Mosiac folder -- FLP folder
mosaic_flp_folder = os.path.join(mosaic_folder , "FLP")
checkPath(mosaic_flp_folder)
# Backfill folder
backfillDir = os.path.join(landscape_folder , "Backfill")
checkPath(backfillDir)
# Backfill folder -- Masks folder
maskDir = os.path.join(backfillDir, "Masks")
checkPath(maskDir)
# Backfill folder -- Zonal folder
zoneDir = os.path.join(backfillDir, "Zonal")
checkPath(zoneDir)
# Final - clipped folder
final_folder = os.path.join(landscape_folder , "Final")
checkPath(final_folder)

arcpy.env.workspace = landscape_folder
arcpy.env.scratchWorkspace = r"D:\ARCTEMP"

My rework of this script would loop through all pyromes instead of selecting the pyromes that intersect with each landscape. This change is resulting in part from the updates to the wcs landscapes, which changes the wcs buffers that the post-processed FSim results are outputted to the extent of. The updated landscape buffers are the AnalysisArea_Buffered_60km.shp. The end result of the script currently is 270-m post-processed FSim outputs by landscape. The update to the script would result in a mosaicked raster of 270-m results, with the extent matching the extent of the entire wcs landscapes buffered.

Conceptual steps:
1. Mosaic all pyromes together? (this could be the first step, or could loop through and then mosaic the results together)
2. Mosaic
3. Backfill
- At some point, we need to use the wcs buffers to mask the process. The end result would be a raster with the extent (rectangle) of the wcs buffers, but only with results for the wcs buffers, and values of NoData filled for the rest of the extent. I need to be mindful of differentiating nodata values that are within the wcs buffers, and nodata values that simply cover the extent.


In [ ]:
wcs_buffers = "C:\Users\Charlie\Desktop\SCRATCH\ArcGIS\OPM_test\data\AnalysisArea\AnalysisArea_Buffered_60km.shp"
lookup_table = 

In [10]:
# Make a table view for getting the number of iterations
arcpy.MakeTableView_management(in_table=iteration_lut_path, out_view = 'lut')

# function to get # of iterations for pyrome
def get_itr(p):
    
    pyxxx = "PY" + p
    
    cursor = arcpy.SearchCursor('lut', fields = "PYROME; ITR")
    
    for row in cursor:
        
        if (row.getValue("PYROME") == pyxxx):
            return row.getValue("ITR")


In [ ]:
# MOSAIC FUNCTIONS

def get_bp_rasters(p):
    
    # Raster object of fsim outputs
    r = Raster(os.path.join(fsim270, "PY{0}_TreatedAllOutputs.tif".format(p)))
    
    # Get the BP band 
    bp = ExtractBand(r, [1])
        
    return (bp)

def get_times_burned(p):
    
    # Raster object of fsim outputs
    r = Raster(os.path.join(fsim270, "PY{0}_TreatedAllOutputs.tif".format(p)))
    
    # Get the number of iterations
    itr = get_itr(str(p))
    
    # Get the BP band 
    bp = ExtractBand(r, [1])
    
    # BP times number of iterations = Count of times burned
    times_burned_in_pyrome = itr * bp
        
    return (times_burned_in_pyrome)

def get_times_flp_burned(p, band):
    
    # Raster object of fsim outputs
    r = Raster(os.path.join(fsim270, "PY{0}_TreatedAllOutputs.tif".format(p)))
    
    # Get the number of iterations
    itr = get_itr(str(p))
    
    # Get the BP band 
    bp = ExtractBand(r, [1])
    
    # Get the FLP band 
    flp = ExtractBand(r, [band])
    
    # BP times number of iterations times flp = Count of times FLP class burned    
    times_flp_burned_in_pyrome = itr * bp * flp
        
    return (times_flp_burned_in_pyrome)

def sum_of_raster_list(this_list):
    
    with arcpy.EnvManager(extent = "MAXOF"):
        
        sum_of_rasters = arcpy.sa.CellStatistics(this_list, "SUM", "DATA")
        
        return(sum_of_rasters)

def x_flp_burned_over_total(flp, total, out_path):
    
    with arcpy.EnvManager(extent = "MAXOF"):
        
        # Calculate final flp:  x times burned within class over total times burned
        final_flp = flp / total
        
        # Fill NoData with 0
        final_flp0 = Con(IsNull(final_flp), 0, final_flp)
        final_flp0.save(out_path)

       

In [7]:
# RUN MOSAIC

# Map functions to get total times burned
list_x_burned = list(map(get_times_burned, pyromes_needed))
x_times_burned = sum_of_raster_list(list_x_burned)
x_times_burned.save(os.path.join(mosaic_folder, "total_times_burned.tif"))

# Map functions to get new bp (sum of all bp rasters)
list_of_bp_rasters = list(map(get_bp_rasters, pyromes_needed))
sum_of_bp_rasters = sum_of_raster_list(list_of_bp_rasters)
sum_of_bp_rasters.save(os.path.join(mosaic_folder, "bp_270m.tif"))

# loop through flps 1 through 6
for i in range(1, 7):
    
    # name (eg. flp1)
    flp_name = "flp" + str(i)
    print(flp_name)
    
    # band index (flp1 is band 2, flp 3 is band 4, etc.)
    band_index = i + 1

    # get a list of rasters that show the number of times a pixel burned within flp
    list_flp_burned = list(map(get_times_flp_burned, pyromes_needed, repeat(band_index)))
    
    # take a sum of the raster list to get the total number of times a pixel burned within flp
    flp_times_burned = sum_of_raster_list(list_flp_burned)
    flp_times_burned.save(os.path.join(mosaic_folder, "x_times_burned_in_{0}.tif".format(flp_name)))
    
    # recalculate the flp.  x times burned in flp / total times burned
    flp_mosaic = os.path.join(mosaic_flp_folder, "{0}_270m.tif".format(flp_name))
    x_flp_burned_over_total(flp_times_burned, x_times_burned, flp_mosaic)

flp1
flp2
flp3
flp4
flp5
flp6


In [8]:
# RUN BACKFILL

# Mosaicked BP
BP = os.path.join(mosaic_folder, "bp_270m.tif")
bp_fill_value = 0.000008

# Directory with FlP rasters at FSim resolution (this is our mosaicked flps)
inFLPdir = mosaic_flp_folder

# Our extent is now the extent of the mosaics
arcpy.env.extent = os.path.join(mosaic_folder, "bp_270m.tif")

# Create the burnable mask from the FM40
print("Starting burnable mask")
FM40ras = arcpy.Raster(fuelsDir + "\\" + FM40)
burnableMask = Con((FM40ras > 99) & (FM40ras < 205), 1, 0)
burnableMaskName = "burnableMask.tif"
arcpy.CopyRaster_management(burnableMask, maskDir + "\\" + burnableMaskName)

# Create the crown-fire-likely mask
print("Starting crown fire likeliness mask")
CCras = arcpy.Raster(fuelsDir + "\\" + CC)
CBHras = arcpy.Raster(fuelsDir + "\\" + CBH)
crownLikelyMask = Con((CCras > 0) & (CBHras <= 30), 1, 0)
crownMaskName = "crownLikelyMask.tif"
arcpy.CopyRaster_management(crownLikelyMask, maskDir + "\\" + crownMaskName)

# Create the FM40-crown fire class raster
print("Starting FM40 and crown fire mask")
FM_crownFire = (FM40ras * 10) + crownLikelyMask
FM_crownFireName = "FM40_crownFireMask.tif"
arcpy.CopyRaster_management(FM_crownFire, maskDir + "\\" + FM_crownFireName)

# Add pyrome identifier to FM40-crown class raster (?)
print("Starting FM40/crown fire/pyromes mask")
pyromesRas = arcpy.Raster(pyromesRaster)
FM_pyrome_crownFire = (FM_crownFire * 100) + pyromesRas
FM_py_CF_name = "FM_pyrome_crownFire.tif"
arcpy.CopyRaster_management(FM_pyrome_crownFire, maskDir + "\\" + FM_py_CF_name)

# Identify pixels that need to be backfilled (burnable but didn't burn)
print("Starting mask of pixels to backfill")
BPras = arcpy.Raster(BP)
noBurnMask = Con((BPras == 0) & (burnableMask == 1), 1, 0)
noBurnMaskName = "burnableNoBurnMask.tif"
arcpy.CopyRaster_management(noBurnMask, maskDir + "\\" + noBurnMaskName)

# Mask for pixels that did burn: for use in calculating average FLPs
print("Starting mask of pixels that burned")
hasBP = Con(BPras > 0, 1, 0)
hasBPname = "hasBP.tif"
arcpy.CopyRaster_management(hasBP, maskDir + "\\" + hasBPname)


# - Calculate zonal averages on the FLP rasters - #

# Make list of FLP rasters
arcpy.env.workspace = inFLPdir
FLPlist = arcpy.ListRasters("*flp*")

# Start loop to produce zonal averages
for FLP in FLPlist:
    print("Starting zonal average on " + FLP)

    # Extract the values to summarize
    FLP_extr = SetNull(hasBP, FLP, "VALUE = 0")

    # Run the zonal statistics tool
    FLPavg = ZonalStatistics(FM_pyrome_crownFire, "VALUE", FLP_extr, "MEAN", "DATA")

    # Export the result
    exportName = FLP[0:-4] + "_zoneAvg.tif"
    exportPath = zoneDir + "\\" + exportName
    arcpy.CopyRaster_management(FLPavg, exportPath)

# - Rescale the FLP values to sum to one (if necessary) - #
# print "\tRescaling zonal averages to a sum of one"
# It doesn't seem like we need to.. at least for the Stanislaus

# - Update the pixels and CLIP - #
print ("\tBackfilling the burnable pixels that didn't burn with zone averages")

# Loop through the FLP rasters (same list as before) and set burnable/non-burned to zonal average
for FLPlyr in FLPlist:
    
    with arcpy.EnvManager(mask = wcs_buffer, extent = wcs_buffer):
        print("Starting backfill for " + FLPlyr)

        # Generate the path to the zone average raster and make raster object
        zoneAvgRasName = FLPlyr[0:-4] + "_zoneAvg.tif"
        zoneAvgRasPath = zoneDir + "\\" + zoneAvgRasName
        zoneAvgRas = arcpy.Raster(zoneAvgRasPath)

        # Set pixels to zone average if burnable but not burned, otherwise keep FLP value
        backfilledFLP = Con(noBurnMask == 1, zoneAvgRas, FLPlyr)

        # Save the output
        outBackfilledName = FLPlyr[0:-4] + "_backfilled.tif"
        outBackfilledPath = final_folder + "\\" + outBackfilledName
        arcpy.CopyRaster_management(backfilledFLP, outBackfilledPath)
        
# Backfill the BP
print("Starting backfill for BP")
with arcpy.EnvManager(mask = wcs_buffer, extent = wcs_buffer):
    backfilledBP = Con(noBurnMask == 1, bp_fill_value, BP)
    
    # Save the output
    outBackfilledName = "bp_270m_backfilled.tif"
    outBackfilledPath = final_folder + "\\" + outBackfilledName
    arcpy.CopyRaster_management(backfilledBP, outBackfilledPath)

#arcpy.CheckInExtension("spatial")
print ("\n")
print ("Completed. Now check that it worked.")

Starting burnable mask
Starting crown fire likeliness mask
Starting FM40 and crown fire mask
Starting FM40/crown fire/pyromes mask
Starting mask of pixels to backfill
Starting mask of pixels that burned
Starting zonal average on flp1_270m.tif
Starting zonal average on flp2_270m.tif
Starting zonal average on flp3_270m.tif
Starting zonal average on flp4_270m.tif
Starting zonal average on flp5_270m.tif
Starting zonal average on flp6_270m.tif
	Backfilling the burnable pixels that didn't burn with zone averages
Starting backfill for flp1_270m.tif
Starting backfill for flp2_270m.tif
Starting backfill for flp3_270m.tif
Starting backfill for flp4_270m.tif
Starting backfill for flp5_270m.tif
Starting backfill for flp6_270m.tif


Completed. Now check that it worked.
